In [12]:
#!pip install Pillow
#!pip install matplotlib

In [ ]:
Estoy intentando crear un script que me facilite la vida a la hora de crear las texturas de un videojuego que estoy haciendo.
Básicamente es un sistema de descomposición donde tengo 3 texturas, skin, muscle y skeleton.
Me gustaría que la piel se empiece a desintegrar y se muestre la carne debajo, luego de unas iteraciones (3 por ejemplo), la carne también se empieza a desintegrar y se ven los huesos.
El esqueleto no se desintegra. Finalmente, antes de la última iteracion queda únicamente el esqueleto. Todo lo demás se remueve. Podrías ayudarme a completar/modificar este código bajo estas especificaciones?

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random

def generar_imagenes_con_heatmap(
    path_original,
    path_heatmap,
    base_probabilidad=0.01,
    umbral=0.2,
    iteraciones=50
):
    # Cargar imágenes
    imagen_original = Image.open(path_original).convert("RGBA")  # soporta transparencia
    heatmap = Image.open(path_heatmap).convert("L")              # blanco y negro

    # Convertir a numpy arrays
    original_np = np.array(imagen_original)
    heatmap_np = np.array(heatmap) / 255.0  # normalizado entre 0 y 1

    imagenes_generadas = []
    imagenes_generadas.append(original_np)

    for i in range(1, iteraciones + 1):
        nueva_imagen = imagenes_generadas[i-1].copy() #original_np.copy()

        for y in range(heatmap_np.shape[0]):
            for x in range(heatmap_np.shape[1]):
                pixel_prob = i * base_probabilidad*random.random() * heatmap_np[y, x]
                if pixel_prob > umbral:
                    nueva_imagen[y, x] = [0, 0, 0, 0]  # transparente

        imagenes_generadas.append(nueva_imagen)
        # Guardar nueva imagen
        img_resultado = Image.fromarray(nueva_imagen)
        img_resultado.save(f"output/iteracion_{i:03d}.png")

    return imagenes_generadas


In [63]:
from PIL import Image
import numpy as np
import random
import os

def load_image(path):
    return np.array(Image.open(path).convert("RGBA"))

def disintegrate_layer(layer, heatmap_np, iteration, start_iter, base_probability, threshold):
    new_layer = layer.copy()
    if iteration < start_iter:
        return new_layer

    for y in range(layer.shape[0]):
        for x in range(layer.shape[1]):
            alpha = layer[y, x, 3]
            if alpha == 0:
                continue  # Already transparent

            pixel_prob = (iteration - start_iter + 1) * base_probability * random.random() * heatmap_np[y, x]
            if pixel_prob > threshold:
                new_layer[y, x] = [0, 0, 0, 0]  # Make transparent

    return new_layer

def combine_layers(skin, muscle, skeleton):
    height, width = skin.shape[:2]
    final = np.zeros((height, width, 4), dtype=np.uint8)

    for y in range(height):
        for x in range(width):
            if skin[y, x, 3] != 0:
                final[y, x] = skin[y, x]
            elif muscle[y, x, 3] != 0:
                final[y, x] = muscle[y, x]
            elif skeleton[y, x, 3] != 0:
                final[y, x] = skeleton[y, x]
            else:
                final[y, x] = [0, 0, 0, 0]  # Fully transparent

    return final

def generate_decomposition(
    path_skin,
    path_muscle,
    path_skeleton,
    path_heatmap,
    iterations=50,
    base_probability=0.3,
    threshold=0.2,
    start_muscle=5,
    output_folder="output",
    prefix=""
):
    os.makedirs(output_folder, exist_ok=True)

    skin_np = load_image(path_skin)
    muscle_np = load_image(path_muscle)
    skeleton_np = load_image(path_skeleton)
    heatmap_np = np.array(Image.open(path_heatmap).convert("L")) / 255.0

    for i in range(1, iterations + 1):
        if i == 1:
            Image.fromarray(skin_np).save(os.path.join(output_folder, f"{0:03d}.png"))
        skin_np = disintegrate_layer(skin_np, heatmap_np, i, start_iter=1, base_probability=base_probability, threshold=threshold)
        muscle_np = disintegrate_layer(muscle_np, heatmap_np, i, start_iter=start_muscle, base_probability=base_probability, threshold=threshold)
        final_np = combine_layers(skin_np, muscle_np, skeleton_np)

        result_img = Image.fromarray(final_np)
        result_img.save(os.path.join(output_folder, f"{prefix}{i:03d}.png"))
        if i == iterations:
            Image.fromarray(skeleton_np).save(os.path.join(output_folder, f"{i+1:03d}.png"))

In [ ]:
titan = "jaw/head"
generate_decomposition(
    titan+"/skin.png",
    titan+"/flesh.png",
    titan+"/skeleton.png",
    titan+"/heatmap.png",
    iterations=9,
    output_folder=titan+"/output",
    prefix="h",
)